In [47]:
import os
if os.path.exists("/content/Captcha_iitkgp") is not True:
  !git clone "https://github.com/Sandeep-1507/Captcha_iitkgp"
  !unzip /content/Captcha_iitkgp/Model/model.zip

In [48]:
import os
import numpy as np
import matplotlib.pyplot as plt
import string
from PIL import Image
from pathlib import Path
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [49]:
from tensorflow.keras.models import Sequential, save_model, load_model
model = load_model("/content/kaggle/working", compile = True)

In [50]:
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)
prediction_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 50, 198, 1)]      0         
                                                                 
 Conv1 (Conv2D)              (None, 50, 198, 32)       320       
                                                                 
 pool1 (MaxPooling2D)        (None, 25, 99, 32)        0         
                                                                 
 Conv2 (Conv2D)              (None, 25, 99, 64)        18496     
                                                                 
 pool2 (MaxPooling2D)        (None, 12, 49, 64)        0         
                                                                 
 reshape (Reshape)           (None, 12, 3136)          0         
                                                                 
 dense1 (Dense)              (None, 12, 64)            2007

In [88]:

characters=set({ 'H', 'E', 'B', '8', 's', '2', 'J', 'K', 'G', 'T', '5', 'F', 'Y', 'U', 'C', 'R', 'L', 'S', 'W', 'Z', 'X', '9', '3', '7', '6', 'A', 'V', 'N', 'w', 'P', 'D', 'u', 'M', '4', 'c', 'x', 'y', 'j''C', 'T', '7', 'R', '5', '4', '2', 'D', 'M', 'G', 'Z', 'X', 'H', 'V', 'W', 'P', 'A', 'U', 'J', 'K', '6', 'B', 'N', 'E', '8', 'Y', '3', '9', 'S', 'F', 'L'})
print(characters)
characters=sorted(list(characters))
char_to_num=layers.StringLookup(vocabulary=list(characters),mask_token=None)
num_to_char = layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)


def decode_batch_predictions(pred):
    max_length=6
    input_len = np.ones(pred.shape[0]) * pred.shape[1]

    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]

    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text




{'Z', 'L', 'T', 'G', 'S', 'A', 'D', 'jC', 'N', 'Y', 'V', 'W', 'c', '9', '6', 'u', 'H', '5', 'J', 'U', 's', 'w', 'x', '4', 'y', '8', 'C', 'P', 'X', '7', 'K', 'R', 'B', 'E', '2', '3', 'F', 'M'}


In [89]:
def captchasolve(folder_path,model,num_images=1):
  data_dir=Path(folder_path)
  images=sorted(list(map(str,list(data_dir.glob("*.jpeg")))))
  labels=[img.split(os.path.sep)[-1].split(".jpeg")[0] for img in images]
  # display(images[0])
  # display(labels[0])


  def encode_single_sample(img_path,label):
    img=tf.io.read_file(img_path)
    img=tf.io.decode_jpeg(img,channels=1)
    img=tf.image.convert_image_dtype(img,tf.float32)
    label=char_to_num(tf.strings.unicode_split(label,input_encoding="UTF-8"))

    return {"image": img, "label": label}

  max_length=6
  batch_size=num_images
  x_test, y_test = (np.array(images), np.array(labels))

  test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
  test_dataset = (test_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE))

  # batch_images = test_dataset.take(1)["image"]
  batch_images=[batch["image"] for batch in test_dataset.take(1)]

  preds = prediction_model.predict(batch_images)
  pred_texts = decode_batch_predictions(preds)
  print(pred_texts)

In [90]:
captchasolve("/content/Captcha_iitkgp/Data/testset",prediction_model,num_images=100)

4/4 [==============================] - 0s 104ms/step
['2AK279', '2CJ763', '2DD7MJ', '2DG53S', '2DWW6A', '2Z7MW[UNK]', '2EP299', '2FV39D', '2FW97C', '2H24TY', '2H6X5B', '2HY4XV', '2J4LZF', '2J76D7', '2JDXR9', '2K7VK6', '2KA575', '2KRR6F', '2KX2GE', '2NR756', '2SW3HH', '2PG536', '2PG573', '2PX3CP', '2PX453', '2SN3P6', '2U3977', '2U726X', '2U7732', '2UPUBS', '2UXD5X', '2V39A3', '2V4G35', '2V5277', '2VU77D', '2X3YR7', '2Y593E', '2YHKT9', '2YXKB7', '2Z875R', '2ZKJPK', '2ZX5PS', '3A38KU', '3A3PH9', '3A77P3', '3ANP72', '3B59UB', '3B8497', '3B87FH', '3C367H', '3CK7XK', '3CP3PS', '3D33U7', '3D4324', '3D54P7', '3D97DA', '3DN34N', '3E98KH', '3EJ6UU', '3EKC6C', '3EU2LL', '3F7P63', '3FN8M4', '3FUGPU', '3G8746', '3G8C9P', '3GG82K', '3GSXNF', '3GVU6Z', '3H955F', '3HJH53', '3HSUMJ', '3HX3XL', '3JHPD2', '3K638G', '3L54XW', '3LJTHH', '3LP2AV', '3LR449', '3M8XP7', '3MRBX8', '3NKADL', '3NLBKJ', '3NXW2Z', '3P3LE5', '3P5P7U', '3P5SUP', '3P629S', '3PE79X', '3PGV36', '3PNMX8', '3PPXEW', '3RV3F7', '3S9EJB', '3